# *N*-grammes et collocations

## Définitions

### *N*-grammes

En traitement automatique du langage naturel, un *n*-gramme représente simplement une séquence de *n* mots. Les unigrammes, bigrammes et tétragrammes sont des cas spécifiques d’association de 1, 2 ou 3 mots quand, au-delà, on parle davantage de 4-grammes, 5-grammes etc. que de tétragrammes ou pentagrammes.

Pour ne s'attacher qu’aux séquences de deux mots, on dénombre cinq bigrammes dans l’énoncé suivant :

```txt
(1) Le petit chat boit du lait.
```

Ces bigrammes sont :
1. Le, petit
2. petit, chat
3. chat, boit
4. boit, du
5. du, lait

Dans cette liste, tous les bigrammes n’ont pas le même poids. Quand *boit, du* ne veut pas dire grand chose, le bigramme *petit, chat* est bien plus significatif. C’est ici qu’entrent en jeu les collocations.

### *Skip-grams*

Le terme de *skip-grams* désigne davantage une méthode de constitution de *n*-grammes qu’une véritable unité linguistique. Dans l’exemple (1), nous avons dénombré cinq bigrammes en tenant compte du contexte immédiat de chaque terme, excluant de fait l’association *chat*, *lait* qui pourtant semblait prometteuse. Pour l’intégrer, il suffit de déterminer au préalable une fenêtre contextuelle suffisante.

### Collocations

Exprimée simplement, la collocation est le processus d’identification de deux ou plusieurs mots qui apparaissent fréquemment ensemble dans un énoncé.

Elle permet de mettre en évidence, dans un énoncé, différents phénomènes linguistiques comme :
- la lexicalisation (*au fur et à mesure*, *c’est-à-dire*)
- les tics de langage (*pas de souci*, *ou pas*, *voilà voilà*)
- les cooccurrences privilégiées (*courir vite*, *procès d’intention*, *soleil de plomb*)

Une cooccurrence est qualifiée de collocation lorsqu’elle apparaît significativement plus souvent que ce que l’on attendrait si ses éléments étaient utilisés indépendamment.

Par exemple, la cooccurrence *nuit noire* est qualifiée de collocation si sa probabilité conjointe $P(\text{nuit} \cap \text{noire})$ est significativement supérieure au produit des probabilités individuelles $P(\text{nuit}) \cdot P(\text{noire})$, c’est-à-dire à la probabilité attendue si les deux mots étaient utilisés indépendamment. Si dans un corpus fictif de 1 000 000 de mots, les mots *nuit* et *noire* apparaissent respectivement 10 000 et 8000 fois et que la cooccurrence *nuit noire* apparaît 1500 fois, alors :

- $P(\text{nuit}) = 0.01$
- $P(\text{noire}) = 0.008$
- $P(\text{nuit} \cap \text{noire}) = 0.0015$
- $P(\text{nuit}) \cdot P(\text{noire}) = 0.00008$

La cooccurrence *nuit noire* est statistiquement plus probable (0,0015) que l’hypothèse d’indépendance (0,00008). Pour juger de sa significativité, il faudrait utiliser un test statistique, comme le $\chi^2$, le *log-likelihood ratio* ou encore le *t-score*. Pour notre exemple, en appliquant la formule du *t-score* :

$$
t = \dfrac{O - E}{\sqrt{O}}
$$

Où :

- *O* représente l’effectif observé ;
- *E* vaut pour l’effectif attendu sous indépendance.

Nous obtenons :

$$
E = \dfrac{10000 \times 8000}{1000000} = 80
$$

$$
t = \dfrac{1500 - 80}{\sqrt{1500}} \approx 36.65
$$

Le *t-score*, bien plus élevé que sous indépendance (0), révèle la significativité de la cooccurrence. À noter qu’en TAL, on considère souvent un seuil $t \gt 2$.

## Lister des *n*-grammes

La librairie NLTK met à disposition des méthodes pour lister facilement les *n*-grammes dans un énoncé. Ces méthodes ayant besoin en entrée d’une liste de mots, une étape préalable de tokenisation est indispensable :

In [ ]:
from nltk.tokenize import word_tokenize

sent = "Le petit chat boit du lait."
words = word_tokenize(sent)

Pour lister les bigrammes, appeler la méthode `.bigrams()` :

In [ ]:
from nltk import bigrams

n_grams = bigrams(words)
list(n_grams)

Même principe pour les trigrammes :

In [ ]:
from nltk import trigrams

n_grams = trigrams(words)
list(n_grams)

Et au-delà ? Une méthode `.ngrams()` avec un paramètre `n` pour définir le *n*-gramme souhaité. Par exemple pour des tétragrammes (4-grammes) :

In [ ]:
from nltk import ngrams

n_grams = ngrams(words, 4)
list(n_grams)

Afin d’améliorer les résultats, supprimer les mots vides et la ponctuation :

In [ ]:
from nltk import bigrams
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# regexp: selects only words
tokenizer = RegexpTokenizer(r'\w+')

# list stopwords
stopwords = stopwords.words('french')

# tokenization
words = [
    word.lower()
    for word in tokenizer.tokenize(sent)
]

On ne sélectionne un bigramme que si aucun de ses éléments ne fait partie des mots vides :

In [ ]:
n_grams = bigrams(words)

clean_bigrams = [
    (w, c)
    for w, c in n_grams
    if w not in stopwords
    and c not in stopwords
]

list(clean_bigrams)

Pour relever des *skip-grams*, NLTK propose la fonction `skipgrams()` dans le module `util` :

In [ ]:
from nltk.util import skipgrams

Les paramètres `n` et `k` permettent de fixer respectivement le degré et la fenêtre contextuelle des *n*-grammes à recenser dans une liste de mots fournie en entrée :

In [ ]:
n_grams = skipgrams(words, n=2, k=3)

clean_bigrams = [
    (w, c)
    for w, c in n_grams
    if w not in stopwords
    and c not in stopwords
]

clean_bigrams

## Détecter des collocations

La manière la plus rapide de détecter des collocations est d’utiliser la méthode `.collocations()` de la classe `Text` appliquée à un texte segmenté en mots :

In [ ]:
from nltk.corpus import PlaintextCorpusReader
from nltk.text import Text

# loading the corpus
corpus = PlaintextCorpusReader('./data', r'.*', encoding='utf8')

# collocations in Salammbô
salammbo = Text(corpus.words('salammbo.txt'))
salammbo.collocations()

### Les *n*-grammes

Les classes `BigramCollocationFinder` et `TrigramCollocationFinder`, permettent de dénicher dans un texte les 2-grammes et 3-grammes qui forment des collocations :

In [ ]:
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

En entrée, ces classes ont besoin qu’on leur fournisse une liste de mots :

In [ ]:
from nltk.corpus import PlaintextCorpusReader

# loading the corpus
corpus = PlaintextCorpusReader('./data', r'.*', encoding='utf8')

# list of words
words = [
    word.lower()
    for word in corpus.words('salammbo.txt')
]

# 2-grams collocation finder
collocations = BigramCollocationFinder.from_words(words)

#### Score des *n*-grammes

Le package `nltk.metrics` offre des outils de mesure adaptés pour attribuer un score aux *n*-grammes :

In [ ]:
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures

Parmi les outils de mesure à disposition, [la fonction de vraisemblance](https://fr.wikipedia.org/wiki/Rapport_de_vraisemblance) (*likelihood ratio*) est souvent la première à utiliser :

In [ ]:
likelihood = BigramAssocMeasures.likelihood_ratio

Et pour obtenir ensuite les *n* bigrammes les plus fréquents, appeler la méthode `.nbest()` :

In [ ]:
collocations.nbest(likelihood, 5)

La méthode `.score_ngrams()` permet de connaître le score attribué aux *n*-grammes :

In [ ]:
collocations.score_ngrams(likelihood)[:5]

Ce qui permet de limiter les résultats aux *n*-grammes qui dépassent un certain score :

In [ ]:
list(collocations.above_score(likelihood, 2000))

En plus de la fonction de vraisemblance, citons quelques autres outils de mesure :
- la fréquence d’apparition
- la [PMI](https://en.wikipedia.org/wiki/Pointwise_mutual_information) (*Pointwise mutual information*)
- le [test de Student](https://fr.wikipedia.org/wiki/Test_de_Student)
- le [test du $χ^2$](https://fr.wikipedia.org/wiki/Test_du_%CF%87%C2%B2)

In [ ]:
print(f"Fréquence d’apparition : {collocations.nbest(BigramAssocMeasures.raw_freq, 5)}")
print(f"PMI : {collocations.nbest(BigramAssocMeasures.pmi, 5)}")
print(f"Test t : {collocations.nbest(BigramAssocMeasures.student_t, 5)}")
print(f"Khi carré : {collocations.nbest(BigramAssocMeasures.chi_sq, 5)}")

#### Filtrer les résultats

Les résultats ne sont pas très probants : ponctuations et mots vides ressortent comme les plus fréquents. Une méthode `.apply_word_filter()` ajoute un filtre sur les mots sélectionnés :

In [ ]:
from nltk.corpus import stopwords

# list of stopwords
stopwords = stopwords.words('french')

# a filter calls a lambda function
filter_stopwords = lambda w: w in stopwords
collocations.apply_word_filter(filter_stopwords)

Même si la situation s’améliore, les signes de ponctuation perturbent encore les résultats :

In [ ]:
collocations.nbest(likelihood, 5)

La solution consiste à modifier le filtre afin de supprimer les mots de un ou deux caractères :

In [ ]:
filter_stopwords = lambda w: w in stopwords or len(w) < 3
collocations.apply_word_filter(filter_stopwords)
collocations.nbest(likelihood, 5)

Il est également possible d’imposer une fréquence d’apparition minimale à un *n*-gramme grâce à la méthode `apply_freq_filter()` :

In [ ]:
collocations.apply_freq_filter(3)
collocations.nbest(likelihood, 5)